In [1]:
import torch
import torch.nn as nn

In [21]:
class MyLinear(nn.Module):

    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__()

        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        y = self.linear(x)
        return y 

In [25]:
objective = 100

x = torch.FloatTensor(16, 10)
linear = MyLinear(10, 5)
y = linear(x)
loss = pow((objective - y.sum()),2) # must be scalar
loss.backward()


In [26]:
# train() and eval()
# 검증후 train()을 통해서 원래 상태로 되돌려줘야한다.
linear.eval()

linear.train()

MyLinear(
  (linear): Linear(in_features=10, out_features=5, bias=True)
)

Linear Regression example

In [34]:
import random 

import torch
import torch.nn as nn 

class MyModel(nn.Module):

    def __init__(self, input_size, output_size):

        super(MyModel, self).__init__()

        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        y = self.linear(x)
        
        return y


def ground_truth(x):
    return 3 * x[:, 0] + x[:, 1] - 2 * x[:, 2]

def train(model, x, y, optim):
    # initialize gradients in all parameters in module.
    optim.zero_grad()

    # feed-forward
    y_hat = model(x)

    # get error between answer and inferenced.
    loss = (pow(y-y_hat, 2).sum() / x.size(0))

    # back-propagation
    loss.backward()

    # one-step of gradient descent
    optim.step()

    return loss.data


batch_size = 1
n_epochs = 1000
n_iter = 10000

model = MyModel(3, 1)
optim = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.1)

for epoch in range(n_epochs):
    avg_loss = 0 

    for i in range(n_iter):
        x = torch.rand(batch_size, 3)
        y = ground_truth(x.data)

        loss = train(model, x, y, optim)

        avg_loss += loss
        avg_loss /= n_iter 

    x_valid = torch.FloatTensor([[.3, .2, .1]])
    y_valid = ground_truth(x_valid.data)

    model.eval()
    y_hat = model(x_valid)
    model.train()


    # if ( epoch % 10 == 0 ):
    #     print(f'epoch : {epoch+1} ; avg_loss : {avg_loss}, y_valid_data[0] : {y_valid.data[0]} ; x_valid_data[0,0] : {x_valid.data[0,0]} ')

    print(avg_loss, y_valid.data[0], x_valid.data[0,0])
    
    if avg_loss < .001:
        break 


tensor(0.0001) tensor(0.9000) tensor(0.3000)


1. nn.Module 클래스를 상속받아 forward 함수를 통해 모델 아키텍처 클래스 선언
2. 해당 클래스 객체 생성
3. SGD나 Adam 등의 optimizer 를 생성하고, 생성한 모델의 파라미터를 최적화 대상으로 등록
4. 데이터로 미니배치를 구성하여 feed-forward 연산 그래프 생성
5. 손실 함수를 통해 최종 결괏값과 손실값 계산
6. 손실에 대해서 backward() 호출 -> 연산 그래프 상의 텐서들의 기울기가 채워짐
7. 3번의 optimizer에서 step()을 호출하여 Gradient Descent 1 step 수행
8. 4번으로 돌아가 수렴 조건이 만족할 때까지 반복 수행